In [11]:
import pefile
import numpy as np

class Rnd521:
    def __init__(self, seed=0):
        self.x = np.zeros(521, dtype=np.uint32)
        
        num = np.uint32(0)
        for i in range(17):
            for _ in range(32):
                seed = np.uint32(seed * 1566083941 + 1)
                num = np.uint32((num >> 1) | (seed & 0x80000000))
            self.x[i] = num

        self.x[16] = (self.x[16] << 23) ^ (self.x[0] >> 9) ^ self.x[15]

        for k in range(17, 521):
            self.x[k] = (self.x[k - 17] << 23) ^ (self.x[k - 16] >> 9) ^ self.x[k - 1]

        self.rnd521()
        self.rnd521()
        self.rnd521()
        self.jrnd = 520

    def rnd521(self):
        for i in range(32):
            self.x[i] ^= self.x[i + 489]
        for i in range(32, 521):
            self.x[i] ^= self.x[i - 32]

    def next(self):
        self.jrnd += 1
        if self.jrnd >= 521:
            self.rnd521()
            self.jrnd = 0
        return self.x[self.jrnd]
    
def decrypt(data:bytearray, offset, length, key):
    rnd = Rnd521(key)
    for i in range(length):
        data[offset + i] ^= rnd.next() & 0xff
        
def uint32_from_bytes(data:bytearray, offset):
    return np.frombuffer(data[offset:offset+4], dtype=np.uint32)[0]

def string_from_bytes(data:bytearray, offset, length):
    return data[offset:offset+length].split(b'\x00', 1)[0]#.decode("ascii")

class PatchInfo:
    def __init__(self):
        self.filename = bytearray()
        self.offset = 0
        self.origin_init_patch_byte = bytearray()
        self.length = 0
        self.patched_init_patch_byte = bytearray()
        self.reserved = 0
        
def load_v4_v5_v6_v7(data):
    array = bytearray(data[:41396])
    decrypt(array, 16, 38208, uint32_from_bytes(array, 8))
    flag = uint32_from_bytes(array, 76)
    filename =string_from_bytes(array, 37968, 256)
    num = 80
    patch = []
    for i in range(256):
        patch_info = PatchInfo()
        patch_info.filename = string_from_bytes(array, num, 128)
        patch_info.offset = uint32_from_bytes(array, num + 128)
        patch_info.origin_init_patch_byte = array[num+132:num+136]
        patch_info.length = uint32_from_bytes(array, num + 136)
        patch_info.patched_init_patch_byte = array[num+140:num+144]
        patch_info.reserved = uint32_from_bytes(array, num + 144)
        num += 148;
        patch.append(patch_info)
        
    return patch

In [12]:
pe_path = 'y:\\Game\\竜姫ぐーたらいふLOVE＋PLUS\\drapriplus_ori.exe'

pe = pefile.PE(pe_path)
overlay = pe.get_overlay()

if overlay[:4] in {b"SdW7", b"SdW6", b"SdW5", b"SdW4"}:
    load_v4_v5_v6_v7(overlay)
elif overlay[:4] in {b"SdW3", b"SdW2"}:
    print("v2-v3")
elif overlay[:4] == b"SdW\0":
    print("v1")
else:
    raise NotImplementedError("This SoftDenchi Wrapper version is not supported, or the executable is not SoftDenchi Wrapper protected.")

In [ ]:
filelist = [
    'y:\\Game\\竜姫ぐーたらいふLOVE＋PLUS\\drapriplus_ori.exe',
    'y:\\Game\\ましろ色シンフォニー SANA EDITION\\mashiro_sana_ori.exe',
    'y:\\Game\\オトメ世界の歩き方\\otomeki_ori.exe',
    'y:\\Game\\セレクトオブリージュ\\selectoblige_ori.exe',
    'y:\\Game\\神様のような君へ\\神様のような君へ Extended Edition ori.exe',
    'y:\\Game\\青春フラジャイル\\cmvs64_ori.exe'
]

for file in filelist:
    pe = pefile.PE(file)
    overlay = pe.get_overlay()
    if not overlay:
        print("No overlay found.")
        continue
    if overlay[:4] in {b"SdW7", b"SdW6", b"SdW5", b"SdW4"}:
        patchlist = load_v4_v5_v6_v7(overlay)
        print(patchlist[0].filename, patchlist[0].offset)
    elif overlay[:4] in {b"SdW3", b"SdW2"}:
        print("v2-v3")
    elif overlay[:4] == b"SdW\0":
        print("v1")
    else:
        raise NotImplementedError("This SoftDenchi Wrapper version is not supported, or the executable is not SoftDenchi Wrapper protected.")

bytearray(b'') 0
No overlay found.
bytearray(b'') 0
bytearray(b'') 0
bytearray(b'') 0
bytearray(b'') 0
bytearray(b'') 0


In [ ]:
#### Ultimately we just needed to do this ####
import pefile
filename = 'y:\\Game\\竜姫ぐーたらいふLOVE＋PLUS\\drapriplus_ori.exe'
pe = pefile.PE(filename)
overlay = pe.get_overlay()
with open(filename.replace(".exe", "_patched.exe"), 'wb') as f:
    f.write(overlay[65536:])
# or even this
with open(filename, "rb") as f:
    data = f.read()
    pe_offset = data.find(b'MZ\x90\x00', 4)
    with open(filename.replace(".exe", "_patched2.exe"), 'wb') as f:
        f.write(data[pe_offset:])